In [18]:
# Mount Google Drive onto Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import numpy
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
from keras.datasets import cifar10
from keras.callbacks import TensorBoard

In [0]:
# this part is used to specify important parameters for TensorBoard such as 
# the batch size and which directory to store the "Event files" which are then used for TensorBoard visualization
tensorboard = TensorBoard(
  log_dir='log/dcgan_log',
  histogram_freq=0,
  batch_size=128,
  write_graph=True,
  write_grads=True
)

In [0]:
# We first develop a generator model which is composed of multiple "Sequential" layers.
# The generator takes in a latent space as input(a Gaussian of Random numbers). We upsample the image consecutively
# into a higher resolution version of the image
def def_gen(latent_dim):
	model = Sequential()
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [0]:
# Now we define a Discriminator in our architecture. The Discriminator takes an input with shape 32*32*3 
# where 32*32 are length and breadth and 3 is the number of color channels(RGB). 
# There are a number of convolutional layers in the model which are then compiled. The loss is set to be "Binary CrossEntropy".
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [0]:
# In order to train the generator, we need to combine the generator and the discriminator into one model.
# The generator model output(the fake images), are set to real(class 1) and fed to the discriminator, 
# Whe the Discriminator classifies the images as fake, the resulting loss is then used to update the Generator weights and train it.
# Note that the Discriminator is set to be "untrainable" when training the Generator because otherwise the Discriminator 
# is trained both independently as well as during the generator training hence making it better 
def define_gan(g_model, d_model):
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [0]:
# linking our combined GAN model to TensorBoard
tensorboard.set_model(define_gan)

In [0]:
# Images from the training set of CIFAR10 are loaded and in order to normalize the images effectively, 
# the input is converted into float values
def load_real_samples():
	(trainX, _), (_, _) = load_data()
	X = trainX.astype('float32')
	X = (X - 127.5) / 127.5
	return X

In [0]:
# Choose at Random values from the Training samples and get the class labels (Y) for the samples
def generate_real_samples(dataset, n_samples):
	ix = randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = ones((n_samples, 1))
	return X, y

In [0]:
# Initially, the generator takes in what is essentially noise as input and generates an images of (32*32*3).
# Points from a Latent vector space are chosen at random 
def generate_latent_points(latent_dim, n_samples):
	x_input = randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [0]:
# Takr the latent space as input and generate fake samples from that
def generate_fake_samples(g_model, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples)
	X = g_model.predict(x_input)
	y = zeros((n_samples, 1))
	return X, y

In [0]:
# scaling of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		new_image = resize(image, new_shape, 0)
		images_list.append(new_image)
	return asarray(images_list)
# Calculate the FID between our 
def calculate_fid(model, images1, images2):
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	covmean = sqrtm(sigma1.dot(sigma2))
	if iscomplexobj(covmean):
		covmean = covmean.real
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [0]:
# Generate a grid plot of size 8x8 and save the plots periodically to drive
def save_plot(examples, epoch, n=8):

  examples = (examples+1)/2.0

  for i in range(n*n):
    pyplot.subplot(n,n,1+i)
    pyplot.axis('off')
    pyplot.imshow(examples[i])

  pyplot.savefig('drive/My Drive/saved_grids/generated_plote%05d.png' % (epoch+1))
  pyplot.close()

#   prepare the two sets of images against which FID is going to be computed. We take images1 from CIFAR10 and take the remaining from the examples we generate   
  model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
  (_,_),(images1, _) = load_data()
  images2 = examples
  shuffle(images1)
  images1 = images1[:1000]
  print('Loaded', images1.shape, images2.shape)
  images1 = images1.astype('float32')
  images1 = (images1 - 127.5) / 127.5
  images2 = images2.astype('float32')
  images1 = scale_images(images1, (299,299,3))
  images2 = scale_images(images2, (299,299,3))
  print('Scaled', images1.shape, images2.shape)

  fid = calculate_fid(model, images1, images2)
  print('FID: %.3f' % fid)

In [0]:
# evaluate the discriminator, plot generated images, save generator model 
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=1500):
	X_real, y_real = generate_real_samples(dataset, n_samples)
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	save_plot(x_fake, epoch)
	filename = 'generator_model_%05d.h5' % (epoch+1)
	g_model.save(filename)


In [0]:
# the train function trains the generator and discrminator by selecting the real images for the discriminator and prepare the input for generator from space
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=5, n_batch=128):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = d_model.train_on_batch(X_real, y_real)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
      X_gan = generate_latent_points(latent_dim, n_batch)
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)      

      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
        (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    tensorboard.on_epoch_end(n_epochs, {"D_real_loss":d_loss1 ,"D_fake_loss":d_loss2,"GAN_loss":g_loss})


    if (i+1) % 2 == 0:
      summarize_performance(i, g_model, d_model, dataset, latent_dim)



In [0]:
# specifying our models(both generator and discriminator)  
latent_dim = 100
d_model = define_discriminator()
g_model = def_gen(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, d_model, gan_model, dataset, latent_dim)